In [2]:
import numpy as np
import cv2
from tensorflow import keras
from keras.optimizers import Adam
from keras.utils import np_utils
from tensorflow.keras.utils import plot_model
#from keras.layers import Activation, Dropout, Convolution2D, GlobalAveragePooling2D
from keras.models import Sequential
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, SeparableConv2D

from keras.layers import Activation, Dropout, Convolution2D, GlobalAveragePooling2D

## Creating Train Dataset :

In [3]:
train_data =r"C:\Users\91968\Desktop\Dive Into DL\Chest X-Ray Images (Pneumonia)\chest_xray\train"

In [4]:
os.listdir(train_data)

['NORMAL', 'PNEUMONIA']

In [5]:
train_df = []

for directory in os.listdir(train_data):
    path = os.path.join(train_data, directory)
    if not os.path.isdir(path):
        continue
    for item in os.listdir(path):
        img = cv2.imread(os.path.join(path, item))
        img = cv2.cvtColor(img,  cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (224, 224))
        train_df.append([img, directory])

In [6]:
y_train=[]
X_train=[]
for x,y in train_df:
    X_train.append(x)
    y_train.append(y)
for i in range(len(y_train)):
    if y_train[i]=='NORMAL':
        y_train[i]=0
    else:
        y_train[i]=1
        
        
X_train=np.array(X_train)
y_train=np.array(y_train)


print(X_train.shape)
print(y_train.shape)

X_train=X_train.reshape(len(X_train),224,224,1)
print('Shape : ' , X_train.shape)

X_train=X_train/255

(5216, 224, 224)
(5216,)
Shape :  (5216, 224, 224, 1)


## Creating Test Dataset :

In [7]:
test_data =r"C:\Users\91968\Desktop\Dive Into DL\Chest X-Ray Images (Pneumonia)\chest_xray\test"

In [8]:
os.listdir(test_data)

['NORMAL', 'PNEUMONIA']

In [9]:
test_df = []

for directory in os.listdir(test_data):
    path = os.path.join(test_data, directory)
    if not os.path.isdir(path):
        continue
    for item in os.listdir(path):
        img = cv2.imread(os.path.join(path, item))
        img = cv2.cvtColor(img,  cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (224, 224))
        test_df.append([img, directory])

In [10]:
y_test=[]
X_test=[]
for x,y in test_df:
    X_test.append(x)
    y_test.append(y)
for i in range(len(y_test)):
    if y_test[i]=='NORMAL':
        y_test[i]=0
    else:
        y_test[i]=1
        
        
X_test=np.array(X_test)
y_test=np.array(y_test)


print(X_test.shape)
print(y_test.shape)

X_test=X_test.reshape(len(X_test),224,224,1)
print('Shape X_test: ' , X_test.shape)

X_test=X_test/255

(624, 224, 224)
(624,)
Shape X_test:  (624, 224, 224, 1)


In [11]:
# Used AlexNet :

def net():
    return tf.keras.models.Sequential([
        
        # input (224*224)
        
        tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=4,input_shape=(224,224,1),
                               activation='relu'),
        tf.keras.layers.BatchNormalization(),
        
        
        #Batch normalization is a technique for training very deep neural networks that standardizes 
        #the inputs to a layer for each mini-batch. This has the effect of stabilizing the learning process 
        #and dramatically reducing the number of training epochs required to train deep networks.
        
        # (224-11+1)/4 
        # (54*54*96)
        
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        # (54-3+1)/2
        #(26*26*96)
        
        tf.keras.layers.Conv2D(filters=256, kernel_size=5, padding='same',
                               activation='relu'),
        tf.keras.layers.BatchNormalization(),
        #(26*26*256)
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        
        #(26-3+1)/2
        #(12*12*256)
        
        tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same',
                               activation='relu'),
        #(12*12*384)
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same',
                               activation='relu'),
        
        #(12*12*384)
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same',
                               activation='relu'),
        
        #(12*12*256)
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        
        # (12-3+1)/2
        # (5*5*256)
        
     
        tf.keras.layers.Flatten(),
        #output from flatten layer : (6400 neurons)
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1)

In [29]:
model=net()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.1),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=[keras.metrics.BinaryAccuracy(name='accuracy')])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 54, 54, 96)        11712     
_________________________________________________________________
batch_normalization_32 (Batc (None, 54, 54, 96)        384       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 26, 26, 256)       614656    
_________________________________________________________________
batch_normalization_33 (Batc (None, 26, 26, 256)       1024      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 12, 12, 384)      

In [39]:
# class_weight = {0: 1,
#                 1: 1}
history=model.fit(X_train,y_train,
                    epochs=10,

                    batch_size=64,
                    #validation_data=(X_test,y_test),
                    #class_weight=class_weight,
                    shuffle=True,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='accuracy',patience=1,mode='auto')]
                    
                 )

Epoch 1/10
82/82 [==============================] - 457s 5s/step - loss: 1.2182 - accuracy: 0.8924
Epoch 2/10
82/82 [==============================] - 391s 5s/step - loss: 0.6640 - accuracy: 0.9254
Epoch 3/10
82/82 [==============================] - 384s 5s/step - loss: 0.4677 - accuracy: 0.9400
Epoch 4/10
82/82 [==============================] - 384s 5s/step - loss: 0.6366 - accuracy: 0.9348


In [51]:
results=model.evaluate(X_test,y_test)

20/20 [==============================] - 10s 511ms/step - loss: 3.2246 - accuracy: 0.7083


In [52]:
results

[3.2246034145355225, 0.7083333134651184]

In [20]:
390/624

0.625

In [19]:
0.6250*624

390.0